In [50]:
# pip install hyperimpute

In [51]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import time

In [52]:
from hyperimpute.plugins.imputers import Imputers

imputers = Imputers()

imputers.list()

['miwae',
 'EM',
 'median',
 'missforest',
 'sinkhorn',
 'most_frequent',
 'mice',
 'hyperimpute',
 'miracle',
 'gain',
 'mean',
 'softimpute',
 'sklearn_missforest',
 'sklearn_ice',
 'nop',
 'ice']

In [53]:
df_mcar = pd.read_parquet('/Users/izadoraganem/GAIN/hyperimpute/bases/df_mcar_50.parquet')
df_copy = pd.read_parquet('//Users/izadoraganem/GAIN/hyperimpute/bases/Base_onda_Onda 1_desfecho_obito_variaveis_all.parquet')
df_copy = df_copy.copy()
df_mcar['sat_fio2'].isna().sum()


3030

In [54]:
def calculate_errors(df_true, df_imputed, column):
    df_subset = df_true.loc[df_imputed.index, :]
    mae = mean_absolute_error(df_subset[column], df_imputed[column])
    mse = mean_squared_error(df_subset[column], df_imputed[column])
    return mae, mse

In [55]:
def GAIN(X):
    method = "gain"

    plugin = Imputers().get(method)
    out = plugin.fit_transform(X.copy())

    return out


In [56]:
import pandas as pd
import numpy as np
import time
from hyperimpute.plugins.imputers import Imputers

imputadores = {'GAIN': GAIN}
metricas = []

for i in range(10):  

    X_train = pd.read_parquet(f'/Users/izadoraganem/Imputation/Imputation/datasets/train_fold_{i}.parquet')
    X_test = pd.read_parquet(f'/Users/izadoraganem/Imputation/Imputation/datasets/test_fold_{i}.parquet')

    X = pd.concat([X_train, X_test], axis=0)

    # print(X)

    for nome, imputador in imputadores.items():
        select = []
        start_time = time.time()
        resultado = imputador(X.copy())
        time_ = time.time() - start_time

        print(f'FOLD {i}')

        # print(resultado)

        resultado = pd.DataFrame(resultado)
        resultado.columns = X_test.columns
        resultado.index = X.index
        
        index = resultado.index
        
        for idx in index:
            if idx in X_test.index:
                select.append(idx)

        print(len(select))
        
        resultado = resultado.loc[select]

        print(resultado.shape)
        # print(resultado_df)
        
        # só calcula o erro entre o que foi imputado por gain 
        mae,mse = calculate_errors(df_copy, resultado, 'sat_fio2')

        metricas.append({'medoto': nome,'fold': i, 'tempo': time_, 'mae': mae, 'mse': mse})
      
# metricas = pd.DataFrame(metricas)  
# print(metricas)


FOLD 0
303
(303, 62)
FOLD 1
303
(303, 62)
FOLD 2
303
(303, 62)
FOLD 3
303
(303, 62)
FOLD 4
303
(303, 62)
FOLD 5
303
(303, 62)
FOLD 6
303
(303, 62)
FOLD 7
303
(303, 62)
FOLD 8
303
(303, 62)
FOLD 9
303
(303, 62)


In [58]:
metricas = pd.DataFrame(metricas)  
# print(metricas)
print(metricas.groupby('medoto').mean())

        fold     tempo        mae           mse
medoto                                         
GAIN     4.5  9.392222  94.506899  18013.318509
